# Query and self healing

In [ ]:

def create_sql_fixing_prompt(table_description, sql_script, error_text):
    template = f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant for data driven company question guideline <|eot_id|><|start_header_id|>user<|end_header_id|>

Here is the data dictionary I have below

Table Details:
{table_description}

Table Relations:
Table1:SALES_TRANSACTION.TRANSACTIONNUMBER [PK] to Table2: SALESTENDER_TRANSACTION.TRANSACTIONNUMBER [PK]
Table1:SALES_TRANSACTION.PLANTCODE [FK] to Table3: PLANTCODE.PLANTCODE [PK]
Table1:SALES_TRANSACTION.MATERIALCODE [FK] to Table4: PRODUCT_DIM.PRODUCTCODE [PK]
Table1:SALES_TRANSACTION.CUSTOMERCODE [FK] to Table5: MCARD_DIM.MCARDCODE [PK]
Table1:SALES_TRANSACTION.STORECODE [FK] to Table7: BRANCH_DIM.BRANCH_COMPANYCODE [PK]
Table5: MCARD_DIM.MEMBERID [FK] to Table6: DEMOGRAPHICS_DIM.MEMBERID [PK]

I Put this SQL script,
SQL:

{sql_script}

But I got this error,
ERROR:

{error_text}

Could you please give me the FIXED SQL based on my situation?
My company have to end the sql script with ;

<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Ok, here is your fixed and improve sql
FIXED SQL:
```
"""
    return template

In [ ]:
with open('temp/sql_script.txt', 'r') as file:
    sql_script = file.read()


error_text = ''
first_sql = sql_script
try:
    print('FIRST TRY SQL: ', first_sql)
    df = idadb.ida_query(first_sql)
    df_pd = pd.DataFrame(df)
    df_pd.to_csv('temp/temp.csv', index = False)        
    file_path = 'temp/temp.csv'
    files = {'csvfile': open(file_path, 'rb')}
    response = requests.post(csv_upload_url, files=files)
    if response.status_code == 200:
        column_number = len(df_pd.columns)
        row_number = len(df_pd)
        result = f"send file successfully without LLM SQL FiXER. This data has {row_number} rows and {column_number} columns, please click refresh to see dashboard"
        result_file = {"results": result}
    else:
        error_message = {"Dashboard Connection denied"}
        result_file = {"results": sample_output}

except Exception as error:
    print(error)
    error_text = str(error)

second_sql = ''
if error_text != '':
    new_sql_prompt = create_sql_fixing_prompt(table_description, first_sql, error_text)
    new_sql_script = send_to_watsonxai(sql_gen_llm_model, [new_sql_prompt])
    second_sql = new_sql_script[0]

if second_sql != '':
    error_text=''
    df = pd.DataFrame()
    print('SECOND TRY SQL: ', second_sql)

    try:
        df = idadb.ida_query(second_sql)

        df_pd = pd.DataFrame(df)
        df_pd.to_csv('temp/temp.csv', index = False)        
        file_path = 'temp/temp.csv'
        files = {'csvfile': open(file_path, 'rb')}
        response = requests.post(csv_upload_url, files=files)
        if response.status_code == 200:
            column_number = len(df_pd.columns)
            row_number = len(df_pd)
            result = f"send file successfully, this time we use LLM SQL FiXER. This data has {row_number} rows and {column_number} columns, please click refresh to see dashboard"
            result_file = {"results": result}
            return result_file
        else:
            error_message = {"Dashboard Connection denied"}
            result_file = {"results": error_message}

    except Exception as error:
        print(f"Error: {error}")
        print(traceback.format_exc())
        sample_output = "QUERY can not run please contact the Data/IT Team"
        result_file = {"results": sample_output}

print(result_file)